<a href="https://colab.research.google.com/github/JessicaaaJe/CardioRenal-Disease-Prediction-using-classification-/blob/main/code_part.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


# **PART A**

In [ ]:
import pandas as pd
import numpy as np
import itertools

In [ ]:
file_path = '/content/drive/MyDrive/share_folders_csc373/Data/Assoc_Analysis_Vidhya.csv'
df = pd.read_csv(file_path)

print(df.head())

   1   Bread    Wine  Eggs    Meat Cheese  Pencil  Diaper
0  2   Bread  Cheese  Meat  Diaper   Wine    Milk  Pencil
1  3  Cheese    Meat  Eggs    Milk   Wine     NaN     NaN
2  4  Cheese    Meat  Eggs    Milk   Wine     NaN     NaN
3  5    Meat  Pencil  Wine     NaN    NaN     NaN     NaN
4  6    Eggs   Bread  Wine  Pencil   Milk  Diaper   Bagel


**Functions to implement the Apriori Algorithm**

In [ ]:
def candidate_generation(frequent_itemsets, k):
    """
    Generate candidate k-itemsets using Fk-1 * Fk-1
    Frequent itemsets is sorted in lexicographic order to avoid duplicates.
    """
    candidate_itemsets = set()
    sorted_frequent_itemsets = sorted(frequent_itemsets)

    for i, itemset1 in enumerate(sorted_frequent_itemsets):
        for itemset2 in sorted_frequent_itemsets[i + 1:]:
            # Check if the first k-1 items are the same
            if list(itemset1)[:k - 1] == list(itemset2)[:k - 1]:
                # Combine if the first k-1 items are the same
                combined = itemset1.union(itemset2)
                candidate_itemsets.add(combined)
    return candidate_itemsets

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
def candidate_prune(candidates, previous_frequent_itemsets, k):
    pruned_candidates = set()
    evious for candidate in candidates:
        # fronzenset is set to make sure the list is hashable, for the later use of checking whether
        # the subset had occured in the prfrequent itemsets.
        # itertools.combinations generate all (k-1)-subsets for the candidate k-itemset
        subsets = [frozenset(subset) for subset in itertools.combinations(candidate, k)]

        # if all its (k-1)-subsets are in the set of frequent (k-1)-itemsets,
        # otherwise eliminiate k itemsets that contain infrequent k-1 itemsets
        if all(subset in previous_frequent_itemsets for subset in subsets):
            pruned_candidates.add(candidate)
    return pruned_candidates

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


`hash tree is used to be more computational efficient`

In [ ]:
def calculate_support(transactions, candidates):
    """
    Use a hash tree for efficient candidate storage and subset checking.
    """
    count = {}
    # Build a hash tree of candidates
    hash_tree = build_hash_tree(candidates)

    for transaction in transactions:
        # Generate all subsets of the transaction that are of the same length as the candidates
        transaction_subsets = generate_subsets(transaction, len(candidates[0]))

        for subset in transaction_subsets:
            if is_subset_in_hash_tree(subset, hash_tree):
                candidate = frozenset(subset)  # Convert to frozenset to use as dictionary key
                count[candidate] = count.get(candidate, 0) + 1

    return count

def is_subset_in_hash_tree(subset, hash_tree):
    """
    Check if a subset is in the hash tree.
    """
    node = hash_tree
    for item in subset:
        index = hash(item)
        if index not in node['children']:
            return False
        node = node['children'][index]
    return True

def generate_subsets(transaction, subset_size):
    """
    Generate all subsets of the transaction that are of the specified size.
    """
    from itertools import combinations
    return [set(s) for s in combinations(transaction, subset_size)]


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
def candidate_elimination(support_counts, transactions, minsup):
    return [itemset for itemset, count in support_counts.items() if count / len(transactions) >= minsup]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


**This is a simple transaction dataset for test.**

In [ ]:
transactions = [
    {'Bread', 'Milk'},
    {'Bread', 'Diapers', 'Beer', 'Eggs'},
    {'Milk', 'Diapers', 'Beer', 'Cola'},
    {'Bread', 'Milk', 'Diapers', 'Beer'},
    {'Bread', 'Milk', 'Diapers', 'Cola'}
]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
minsup = 0.6
print(f"number of transactions:{len(transactions)}")
print(f"minsup:{minsup}\n")
all_frequent_itemsets = []

# create initial candidate 1-itemsets
unique_items = set(item for transaction in transactions for item in transaction)
current_candidates = [frozenset([item]) for item in unique_items]


k = 1
while current_candidates:
    counts = calculate_support(transactions, current_candidates)

    # eliminate infrequent itemsets
    frequent_itemsets = candidate_elimination(counts, transactions, minsup)
    print(f"Frequent {k}-itemsets: {[set(itemset) for itemset in frequent_itemsets]}")

    # If no frequent itemsets, break the loop
    if not frequent_itemsets:
        break

    all_frequent_itemsets.extend(frequent_itemsets)

    # Generate (k+1)-itemsets
    next_candidates = candidate_generation(frequent_itemsets, k)
    print(f"\nCandidate {k+1}-itemsets before pruning:{[set(itemset) for itemset in next_candidates]}")

    # Prune candidates that contain infrequent subsets
    next_candidates = candidate_prune(next_candidates, set(frequent_itemsets), k)
    print(f"Candidate {k+1}-itemsets after pruning: {next_candidates}")

    # Set up for next iteration
    current_candidates = next_candidates
    k += 1

print("\nFinal Frequent Itemsets:")
for itemset in all_frequent_itemsets:
    support_ratio = sum(1 for t in transactions if itemset.issubset(t)) / len(transactions)
    print(f"Itemset: {set(itemset)}, Support: {support_ratio:.3f}")

number of transactions:5
minsup:0.6

Frequent 1-itemsets: [{'Bread'}, {'Milk'}, {'Diapers'}, {'Beer'}]

Candidate 2-itemsets before pruning:[{'Beer', 'Milk'}, {'Bread', 'Milk'}, {'Diapers', 'Beer'}, {'Bread', 'Diapers'}, {'Bread', 'Beer'}, {'Diapers', 'Milk'}]
Candidate 2-itemsets after pruning: {frozenset({'Beer', 'Milk'}), frozenset({'Bread', 'Milk'}), frozenset({'Diapers', 'Beer'}), frozenset({'Bread', 'Diapers'}), frozenset({'Bread', 'Beer'}), frozenset({'Diapers', 'Milk'})}
Frequent 2-itemsets: [{'Bread', 'Milk'}, {'Bread', 'Diapers'}, {'Diapers', 'Beer'}, {'Diapers', 'Milk'}]

Candidate 3-itemsets before pruning:[{'Diapers', 'Beer', 'Milk'}, {'Bread', 'Diapers', 'Milk'}]
Candidate 3-itemsets after pruning: {frozenset({'Bread', 'Diapers', 'Milk'})}
Frequent 3-itemsets: []

Final Frequent Itemsets:
Itemset: {'Bread'}, Support: 0.800
Itemset: {'Milk'}, Support: 0.800
Itemset: {'Diapers'}, Support: 0.800
Itemset: {'Beer'}, Support: 0.600
Itemset: {'Bread', 'Milk'}, Support: 0.600
Ite

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


***The algorithm has successfully beeen implemented. ***


In [ ]:
# Convert the DataFrame into a list of sets (transactions)
transactions = df.apply(lambda row: set(row.dropna().values), axis=1).tolist()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
minsup = 0.2
print(f"number of transactions:{len(transactions)}")
print(f"minsup:{minsup}\n")
all_frequent_itemsets = []

# create initial candidate 1-itemsets
unique_items = set(item for transaction in transactions for item in transaction)
current_candidates = [frozenset([item]) for item in unique_items]


k = 1
while current_candidates:
    counts = calculate_support(transactions, current_candidates)

    # eliminate infrequent itemsets
    frequent_itemsets = candidate_elimination(counts, transactions, minsup)
    print(f"Frequent {k}-itemsets: {[set(itemset) for itemset in frequent_itemsets]}")

    # If no frequent itemsets, break the loop
    if not frequent_itemsets:
        break

    all_frequent_itemsets.extend(frequent_itemsets)

    # Generate (k+1)-itemsets
    next_candidates = candidate_generation(frequent_itemsets, k)
    print(f"\nCandidate {k+1}-itemsets before pruning:{[set(itemset) for itemset in next_candidates]}")

    # Prune candidates that contain infrequent subsets
    next_candidates = candidate_prune(next_candidates, set(frequent_itemsets), k)
    print(f"Candidate {k+1}-itemsets after pruning: {next_candidates}")

    # Set up for next iteration
    current_candidates = next_candidates
    k += 1

# output frequent itemsets in descending order of support ratio
support_ratios = [(itemset, sum(1 for t in transactions if itemset.issubset(t)) / len(transactions)) for itemset in all_frequent_itemsets]
sorted_frequent_itemsets = sorted(support_ratios, key=lambda x: x[1], reverse=True)
print("\nFinal Frequent Itemsets:")
for index, (itemset, support_ratio) in enumerate(sorted_frequent_itemsets,start =1):
    print(f"{index}. Itemset: {set(itemset)}, Support: {support_ratio:.3f}")

number of transactions:314
minsup:0.2

Frequent 1-itemsets: [{'Wine'}, {'Meat'}, {'Milk'}, {'Pencil'}, {'Cheese'}, {'Diaper'}, {'Bread'}, {'Eggs'}, {'Bagel'}]

Candidate 2-itemsets before pruning:[{'Wine', 'Diaper'}, {'Bagel', 'Cheese'}, {'Cheese', 'Meat'}, {'Pencil', 'Cheese'}, {'Pencil', 'Eggs'}, {'Cheese', 'Diaper'}, {'Wine', 'Meat'}, {'Bread', 'Meat'}, {'Bagel', 'Diaper'}, {'Meat', 'Eggs'}, {'Bagel', 'Milk'}, {'Wine', 'Milk'}, {'Pencil', 'Meat'}, {'Bread', 'Diaper'}, {'Meat', 'Diaper'}, {'Wine', 'Cheese'}, {'Cheese', 'Milk'}, {'Eggs', 'Milk'}, {'Bagel', 'Meat'}, {'Bread', 'Bagel'}, {'Pencil', 'Diaper'}, {'Bread', 'Cheese'}, {'Eggs', 'Diaper'}, {'Pencil', 'Bagel'}, {'Meat', 'Milk'}, {'Bagel', 'Eggs'}, {'Bread', 'Milk'}, {'Pencil', 'Milk'}, {'Wine', 'Bread'}, {'Cheese', 'Eggs'}, {'Diaper', 'Milk'}, {'Wine', 'Bagel'}, {'Wine', 'Eggs'}, {'Bread', 'Eggs'}, {'Wine', 'Pencil'}, {'Pencil', 'Bread'}]
Candidate 2-itemsets after pruning: {frozenset({'Wine', 'Diaper'}), frozenset({'Bagel', 'Ch

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# **PART B**

In [ ]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import association_rules
from mlxtend.frequent_patterns import association_rules

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
#  all items are converted to strings to ensure consistency in fitting FP Tree
transactions_list = df.apply(lambda row: row.dropna().astype(str).tolist(), axis=1).tolist()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# encode the transaction into a binary matrix
# where each row represents a transaction and each column represents a unique item

te = TransactionEncoder()
te_ary = te.fit(transactions_list).transform(transactions_list)
ohe_df = pd.DataFrame(te_ary, columns=te.columns_)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
min_support = 0.2
frequent_itemsets_fp = fpgrowth(ohe_df, min_support=min_support, use_colnames=True)

# Sort the frequent itemsets by support in descending order
sorted_frequent_itemsets_fp = frequent_itemsets_fp.sort_values(by='support', ascending=False).reset_index(drop=True)

print("Sorted frequent itemsets from FP-Growth:")
for index, row in sorted_frequent_itemsets_fp.iterrows():
    print(f"{index}. Itemset: {sorted(list(row['itemsets']))}, Support: {row['support']:.3f}")

Sorted frequent itemsets from FP-Growth:
0. Itemset: ['Milk'], Support: 0.503
1. Itemset: ['Bread'], Support: 0.503
2. Itemset: ['Cheese'], Support: 0.500
3. Itemset: ['Meat'], Support: 0.475
4. Itemset: ['Wine'], Support: 0.436
5. Itemset: ['Eggs'], Support: 0.436
6. Itemset: ['Bagel'], Support: 0.427
7. Itemset: ['Diaper'], Support: 0.404
8. Itemset: ['Pencil'], Support: 0.360
9. Itemset: ['Cheese', 'Meat'], Support: 0.322
10. Itemset: ['Cheese', 'Milk'], Support: 0.306
11. Itemset: ['Cheese', 'Eggs'], Support: 0.296
12. Itemset: ['Bread', 'Milk'], Support: 0.280
13. Itemset: ['Bagel', 'Bread'], Support: 0.280
14. Itemset: ['Cheese', 'Wine'], Support: 0.268
15. Itemset: ['Eggs', 'Meat'], Support: 0.264
16. Itemset: ['Meat', 'Wine'], Support: 0.248
17. Itemset: ['Eggs', 'Milk'], Support: 0.245
18. Itemset: ['Meat', 'Milk'], Support: 0.245
19. Itemset: ['Bread', 'Wine'], Support: 0.242
20. Itemset: ['Eggs', 'Wine'], Support: 0.239
21. Itemset: ['Bread', 'Cheese'], Support: 0.236
22. It

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


***The frequent patterns generated from Apriori algorithm and FP Tree are the same. There are in total 28 frequent patterns generated given the minimum support = 0.2. ***

# **PART C**

Interesting rules will be disucssed in the report.

In [ ]:
rules = association_rules(sorted_frequent_itemsets_fp, metric="confidence", min_threshold=0.2)

# Convert frozensets to sorted lists for display
rules['antecedents'] = rules['antecedents'].apply(lambda a: sorted(list(a)))
rules['consequents'] = rules['consequents'].apply(lambda c: sorted(list(c)))

print("Association Rules:")
for index, rule in rules.iterrows():
    print(f"Rule {index + 1}: {rule['antecedents']} => {rule['consequents']}, "
          f"Confidence: {rule['confidence']:.3f}, "
          f"Support: {rule['support']:.3f}, "
          f"Lift: {rule['lift']:.3f}")

Association Rules:
Rule 1: ['Cheese'] => ['Meat'], Confidence: 0.643, Support: 0.322, Lift: 1.356
Rule 2: ['Meat'] => ['Cheese'], Confidence: 0.678, Support: 0.322, Lift: 1.356
Rule 3: ['Cheese'] => ['Milk'], Confidence: 0.611, Support: 0.306, Lift: 1.215
Rule 4: ['Milk'] => ['Cheese'], Confidence: 0.608, Support: 0.306, Lift: 1.215
Rule 5: ['Cheese'] => ['Eggs'], Confidence: 0.592, Support: 0.296, Lift: 1.358
Rule 6: ['Eggs'] => ['Cheese'], Confidence: 0.679, Support: 0.296, Lift: 1.358
Rule 7: ['Bread'] => ['Milk'], Confidence: 0.557, Support: 0.280, Lift: 1.107
Rule 8: ['Milk'] => ['Bread'], Confidence: 0.557, Support: 0.280, Lift: 1.107
Rule 9: ['Bread'] => ['Bagel'], Confidence: 0.557, Support: 0.280, Lift: 1.305
Rule 10: ['Bagel'] => ['Bread'], Confidence: 0.657, Support: 0.280, Lift: 1.305
Rule 11: ['Wine'] => ['Cheese'], Confidence: 0.613, Support: 0.268, Lift: 1.226
Rule 12: ['Cheese'] => ['Wine'], Confidence: 0.535, Support: 0.268, Lift: 1.226
Rule 13: ['Meat'] => ['Eggs'], C

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
